In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
import matplotlib.colors as colors
import pandas as pd
from scipy.stats import norm, chi2
from tqdm import tqdm


# NEW: we will use ks_2samp for a simple statistical test on Kd distributions
from scipy.stats import ks_2samp

# define matplotlib style
mpl.style.use("classic")
mpl.rc("xtick", labelsize=15)
mpl.rc("ytick", labelsize=15)
mpl.rc("xtick.major", size=14, width=2)
mpl.rc("xtick.minor", size=7, width=2, visible=True)
mpl.rc("ytick.major", size=14, width=2)
mpl.rc("ytick.minor", size=7, width=2, visible=True)
mpl.rc("lines", linewidth=2, markersize=5)
mpl.rc("axes", linewidth=2, labelsize=15, labelpad=2.5)
mpl.rc("legend", fontsize=15, loc="best", frameon=True, numpoints=1)

mpl.rc("font", family="STIXGeneral")
mpl.rc("mathtext", fontset="stix")

In [ ]:
import csv
import pandas as pd
from datetime import datetime

INPUT_FILE = 'FileC004.txt'   
ALL_OUT   = 'all_events_with_toggle.csv'
COIN_OUT  = 'coincident_events_with_toggle.csv'

records = []
with open(INPUT_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=None, skipinitialspace=True)
    for row in reader:
        if not row or row[0].startswith('#'):
            continue
        # ensure we have at least 11 columns
        if len(row) < 11:
            continue

        # parse fields
        event_time = row[1].strip()       # e.g. "0:00:07"
        event_date = row[2].strip()       # e.g. "1/1/2019"
        ts_ms      = int(row[3])          # TimeStamp[ms]
        coin       = int(row[10])         # Coincident flag

        # compute toggle bit
        toggle = ts_ms % 2

        records.append({
            'Date':           event_date,
            'Time':           event_time,
            'TimeStamp_ms':   ts_ms,
            'toggle':         toggle,
            'Coincident':     coin,
        })

# build DataFrame
df = pd.DataFrame(records)

# reconstruct full datetime and compute diffs
# assume Date in M/D/YYYY and Time in H:M:S
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'],
                                format='%m/%d/%Y %H:%M:%S')

# sort just in case
df.sort_values('DateTime', inplace=True)
df.reset_index(drop=True, inplace=True)

# Δ from Timestamp_ms
df['Δts_ms'] = df['TimeStamp_ms'].diff()

# Δ from actual datetime (in ms)
df['Δdt_ms'] = df['DateTime'].diff().dt.total_seconds() * 1000

# consistency check: flag large mismatches
df['ts_match'] = (df['Δts_ms'] - df['Δdt_ms']).abs() < 1.0  # allow ±1 ms

# save all events
df.to_csv(ALL_OUT, index=False)
print(f"Wrote {len(df)} events to {ALL_OUT}")

# save only coincidences
df_coin = df[df['Coincident'] == 1]
df_coin.to_csv(COIN_OUT, index=False)
print(f"Wrote {len(df_coin)} coincident events to {COIN_OUT}")

In [ ]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
from scipy import stats
import glob
import os

# --- 0) Define your mapping: filename → angle (in degrees) ------------
# e.g. {'coin_0deg.csv': 0, 'coin_30deg.csv': 30, ...}
file_to_angle = {
    'coincidence_0deg.csv':   0,
    'coincidence_15deg.csv':  15,
    'coincidence_30deg.csv':  30,
    'coincidence_45deg.csv':  45,
    'coincidence_60deg.csv':  60,
    'coincidence_75deg.csv':  75,
    'coincidence_90deg.csv':  90,
    # add as many as you have...
}

# --- 1) Load all‑events toggle bits ----------------------------------
all_df = pd.read_csv('all_events_with_toggle.csv')             # contains 'toggle'
bits = all_df['toggle'].astype(int).values
n = len(bits)

# --- 2) Basic randomness tests on all‑events ------------------------
# Monobit
ones = bits.sum()
p_hat = ones / n
z = (p_hat - 0.5) / np.sqrt(0.25 / n)
p_val_freq = 2 * (1 - stats.norm.cdf(abs(z)))
print(f"Monobit: 1s={ones}/{n}, p̂={p_hat:.4f}, p‑value={p_val_freq:.3f}")

# Runs
runs = 1 + np.sum(bits[1:] != bits[:-1])
mu_runs = 2*n*p_hat*(1-p_hat) + 1
var_runs = (2*n*p_hat*(1-p_hat)*(2*n*p_hat*(1-p_hat)-n*p_hat*(1-p_hat)))/(n**2*(n-1))
z_runs = (runs - mu_runs) / np.sqrt(var_runs)
p_val_runs = 2 * (1 - stats.norm.cdf(abs(z_runs)))
print(f"Runs: runs={runs}, E[runs]={mu_runs:.1f}, p‑value={p_val_runs:.3f}")

# Serial (2‑bit)
pairs = bits.reshape(-1, 2)
pair_vals = pairs[:,0]*2 + pairs[:,1]
counts = np.bincount(pair_vals, minlength=4)
expected = len(pair_vals)/4
chi2_serial = ((counts-expected)**2/expected).sum()
p_val_serial = 1 - stats.chi2.cdf(chi2_serial, df=3)
print(f"Serial: counts={counts.tolist()}, p‑value={p_val_serial:.3f}")

# Lag‑1 autocorrelation
autocorr1 = np.corrcoef(bits[:-1], bits[1:])[0,1]
print(f"Lag‑1 autocorr: {autocorr1:.4f}")

# Byte χ²
m = (n//8)*8
bytes_arr = np.packbits(bits[:m].reshape(-1,8), axis=1).flatten()
freqs = np.bincount(bytes_arr, minlength=256)
chi2_bytes = ((freqs - m/256)**2/(m/256)).sum()
p_val_bytes = 1 - stats.chi2.cdf(chi2_bytes, df=255)
print(f"Byte χ²: p‑value={p_val_bytes:.3f}")

# --- 3) Build a combined coin_df with angles -------------------------
coin_frames = []
for fname, angle in file_to_angle.items():
    if not os.path.exists(fname):
        print(f"Warning: {fname} not found, skipping.")
        continue
    df = pd.read_csv(fname)                          # JLExp56.pdf](file-service://file-TZRdpPZV1dpyvPH7vCuSsM)
    df['angle'] = angle                              # attach angle
    coin_frames.append(df[['toggle','angle']])

if not coin_frames:
    raise RuntimeError("No coincidence files loaded: check your file_to_angle mapping.")

coin_df = pd.concat(coin_frames, ignore_index=True)
angles  = coin_df['angle'].values
toggles = coin_df['toggle'].astype(int).values
N_coin  = len(angles)

# --- 4) Angle vs toggle independence tests ---------------------------
# 4a) Point‑biserial correlation
r_pb, p_pb = stats.pointbiserialr(toggles, angles)
print(f"Point‑biserial corr: r={r_pb:.4f}, p‑value={p_pb:.3f}")

# 4b) χ² test of independence (bin angles)
bins = np.linspace(0, 360, 13)  # 12 bins of 30°
angle_bin = pd.cut(angles, bins, include_lowest=True)
cont = pd.crosstab(angle_bin, toggles)
chi2_ind, p_ind, dof, exp = stats.chi2_contingency(cont)
print(f"Chi‑sq indep: χ²={chi2_ind:.1f}, p‑value={p_ind:.3f}, dof={dof}")

print("\nContingency table (angle_bin × toggle):")
print(cont)

In [1]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
import scipy.stats as stats

# ─── 1) Load data ────────────────────────────────────────────────────────────
all_df  = pd.read_csv('all_events_with_toggle.csv')             # JLExp56.pdf](file-service://file-TZRdpPZV1dpyvPH7vCuSsM)
file_to_angle = {
    'coincidence_0deg.csv':   0,
    'coincidence_30deg.csv': 30,
    'coincidence_60deg.csv': 60,
    'coincidence_90deg.csv': 90,
    # …etc.
}
coin_frames = []
for fname, angle in file_to_angle.items():
    df = pd.read_csv(fname)                                     # JLExp56.pdf](file-service://file-TZRdpPZV1dpyvPH7vCuSsM)
    df['angle'] = angle
    coin_frames.append(df[['toggle','angle']])
coin_df = pd.concat(coin_frames, ignore_index=True)

bits    = all_df['toggle'].astype(int).values
toggles = coin_df['toggle'].astype(int).values
angles  = coin_df['angle'].values

# ─── 2) Bit‑value Histogram ─────────────────────────────────────────────────
plt.figure()
counts = [np.sum(bits==0), np.sum(bits==1)]
plt.bar([0,1], counts, width=0.4)
plt.xticks([0,1])
plt.xlabel('Bit value')
plt.ylabel('Count')
plt.title('Monobit Histogram')
plt.savefig('plot_monobit_histogram.png', dpi=150)

# ─── 3) Running Frequency over Time ──────────────────────────────────────────
plt.figure()
cum_frac = np.cumsum(bits) / np.arange(1, len(bits)+1)
plt.plot(cum_frac)
plt.axhline(0.5, linestyle='--', label='0.5')
plt.xlabel('Event index')
plt.ylabel('Cumulative fraction of 1’s')
plt.title('Cumulative Frequency of 1’s')
plt.legend()
plt.savefig('plot_running_frequency.png', dpi=150)

# ─── 4) Run‑Length Distribution ──────────────────────────────────────────────
# compute run lengths
run_vals = np.split(bits, np.where(bits[:-1] != bits[1:])[0]+1)
run_lengths = [len(r) for r in run_vals]
plt.figure()
plt.hist(run_lengths, bins=50)
plt.xlabel('Run length')
plt.ylabel('Frequency')
plt.title('Run‑Length Distribution')
plt.savefig('plot_run_length.png', dpi=150)

# ─── 5) Autocorrelation Function ────────────────────────────────────────────
plt.figure()
ac = acf(bits, nlags=50, fft=True)
plt.stem(range(len(ac)), ac, use_line_collection=True)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Bitstream ACF')
plt.savefig('plot_autocorrelation.png', dpi=150)

# ─── 6) Serial 2‑Bit Frequencies ─────────────────────────────────────────────
pairs = bits[:(len(bits)//2)*2].reshape(-1,2)
pair_vals = pairs[:,0]*2 + pairs[:,1]
labels = ['00','01','10','11']
counts2 = np.bincount(pair_vals, minlength=4)
plt.figure()
plt.bar(labels, counts2)
plt.xlabel('2‑bit tuple')
plt.ylabel('Count')
plt.title('Serial (2‑bit) Frequencies')
plt.savefig('plot_serial_frequencies.png', dpi=150)

# ─── 7) Toggle–Angle Scatter Plot ────────────────────────────────────────────
plt.figure()
y_jitter = toggles + (np.random.rand(len(toggles)) - 0.5)*0.1
plt.scatter(angles, y_jitter, s=5, alpha=0.3)
plt.yticks([0,1])
plt.xlabel('Angle (deg)')
plt.ylabel('Toggle bit (jittered)')
plt.title('Toggle vs Angle Scatter')
plt.savefig('plot_toggle_angle_scatter.png', dpi=150)

# ─── 8) Toggle‑Rate by Angle‑Bin ─────────────────────────────────────────────
bins = np.linspace(0,360,13)
df = pd.DataFrame({'angle': angles, 'toggle': toggles})
grp = df.groupby(pd.cut(df.angle, bins)).toggle.mean()
plt.figure()
grp.plot.bar()
plt.ylabel('P(toggle=1)')
plt.xlabel('Angle bin')
plt.title('Toggle‑Rate by Angle Bin')
plt.tight_layout()
plt.savefig('plot_toggle_rate_by_angle_bin.png', dpi=150)

# ─── 9) Inter‑arrival Times Histogram & Exponential Fit ─────────────────────
# need Δts_ms from all_df (you saved it previously)
dt = all_df['Δts_ms'].dropna().values
lam = 1/np.mean(dt)  # estimated rate (per ms)
plt.figure()
plt.hist(dt, bins=50, density=True, alpha=0.6)
x = np.linspace(0, dt.max(), 300)
plt.plot(x, lam * np.exp(-lam*x), label=f'Exp(λ={lam:.3f} ms⁻¹)')
plt.xlabel('Δts (ms)')
plt.ylabel('Density')
plt.title('Inter‑arrival Time Histogram')
plt.legend()
plt.savefig('plot_interval_histogram.png', dpi=150)

# ─── 10) Q–Q Plot vs Exponential ─────────────────────────────────────────────
plt.figure()
stats.probplot(dt, dist=stats.expon(scale=1/lam), plot=plt)
plt.title('Q–Q Plot vs Exponential')
plt.savefig('plot_qq_exponential.png', dpi=150)

print("All plots saved as PNGs.")  

UnicodeEncodeError: 'latin-1' codec can't encode character '\u273f' in position 74: ordinal not in range(256)